<a href="https://colab.research.google.com/github/Luandersonsouza/Missionaries_Cannibals/blob/main/Missionarios_Canibais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook criado com intuito de solucionar o problema proposto em aula.

In [ ]:
"""                         Missionarios e Canibais
      Problema consiste em atravessar três missionários e três canibais
      de uma margem do rio para a outra.
"""

In [48]:
class State():

    def __init__(self, missionaries_left, missionaries_right , cannibals_left, cannibals_right, river_side):
        """
            Inicializa um estado com as informações de quantidade de missionários e canibais de
            cada lado do rio, além da informação de em que lado do rio está o barco.
        """
        self.missionaries_left = missionaries_left
        self.missionaries_right = missionaries_right
        self.cannibals_left = cannibals_left
        self.cannibals_right = cannibals_right
        self.river_side = river_side
        self.father = None
        self.son = []

    def __str__(self):
        """
            Define a representação em string de um estado.
        """
        return 'Missionaries: {}\t| Missionaries: {}\n Cannibals: {}\t| Cannibals: {}'.format(
            self.missionaries_left, self.missionaries_right, self.cannibals_left, self.cannibals_right
        )

    def valide_state(self):
        """
            Verifica se o estado é válido, ou seja, não possue mais canibais que missionários
            em nenhum lado do rio.
        """
        # Não se pode gerar estados onde o número de canibais ou missionários em qualquer lado
        # do rio seja negativo
        if ((self.missionaries_left < 0) or (self.missionaries_right < 0)
            or (self.cannibals_left< 0) or (self.cannibals_right < 0)):
            return False
        # Verifica se em ambas as margens do rio o número de missionários não é inferior ao número
        # de canibais. Lembrando que caso não hajam missionários em um dos lados, não é necessário
        # verificar o número de canibais nele.
        return ((self.missionaries_left == 0 or self.missionaries_left >= self.cannibals_left) and
                (self.missionaries_right == 0 or self.missionaries_right >= self.cannibals_right))


    def last_state(self):
        """
            Verifica se o estado é um estado final, ou seja, é uma das possíveis soluções do
            problema.
        """
        # Um estado é um estado final se todos os missionários e canibais atravessaram o rio
        result_left = self.missionaries_left == self.cannibals_left == 0
        result_right = self.missionaries_right == self.cannibals_right == 3
        return result_left and result_right

    def make_children(self):
        """
            Gera todos os possíveis filhos de um estado, se este for um estado válido e não
            for um estado final.
        """
        # Encontra o novo lado do rio
        new_river_side = 'right' if self.river_side == 'left' else 'left'
        # Gera a lista de possíveis movimentos
        movements = [
            {'missionaries': 2, 'cannibals': 0},
            {'missionaries': 1, 'cannibals': 0},
            {'missionaries': 1, 'cannibals': 1},
            {'missionaries': 0, 'cannibals': 1},
            {'missionaries': 0, 'cannibals': 2},
        ]
        # Gera todos os possíveis estados e armazena apenas os válidos na lista de filhos
        # do estado atual
        for moviment in movements:
            if self.river_side == 'left':
                # Se o barco estiver a esquerda do rio, os missionários e canibais saem da
                # margem esquerda do rio e vão para a direita
                missionaries_left = self.missionaries_left - moviment['missionaries']
                missionaries_right = self.missionaries_right + moviment['missionaries']
                cannibals_left = self.cannibals_left - moviment['cannibals']
                cannibals_right = self.cannibals_right + moviment['cannibals']
            else:
                # Caso contrário, os missionários e canibais saem da margem direita do rio
                # e vão para a esquerda
                missionaries_right = self.missionaries_right - moviment['missionaries']
                missionaries_left = self.missionaries_left + moviment['missionaries']
                cannibals_right = self.cannibals_right - moviment['cannibals']
                cannibals_left = self.cannibals_left + moviment['cannibals']
            # Cria o estado do filho e caso este seja válido, o adiciona à lista de filhos do pai
            children = State(missionaries_left, missionaries_right, cannibals_left,
                           cannibals_right, new_river_side)
            children.father = self
            if children.valide_state():
                self.son.append(children)


In [49]:
class Missionarios_Canibais():
    """
        Resolve o problema dos missionários e canibais, gerando para isso uma árvore de estados.
    """

    def __init__(self):
        """
            Inicializa uma instância do problema com uma raiz pré-definida e ainda sem solução.
        """
        # Insere a raiz na fila de execução, que será utilizada para fazer uma busca em largura
        self.fila_execucao = [State(3, 0, 3, 0, 'left')]
        self.solucao = None

    def gerar_solucao(self):
        """
            Encontra a solução gerando uma árvore de estados a ser percorrida com o algoritmo de
            busca em largura, que utiliza uma fila em sua execução.
        """
        # Realiza a busca em largura em busca da solução
        for elemento in self.fila_execucao:
            if elemento.last_state():
                # Se a solução foi encontrada, o caminho que compõe a solução é gerado realizando
                # o caminho de volta até a raiz da árvore de estados e então encerra a busca
                self.solucao = [elemento]
                while elemento.father:
                    self.solucao.insert(0, elemento.father)
                    elemento = elemento.father
                break;
            # Caso o elemento não seja a solução, gera seus filhos e os adiciona na fila de execução
            elemento.make_children()
            self.fila_execucao.extend(elemento.son)

In [ ]:
def main():
    # Instancia o problema e gera sua solução
    problema = Missionarios_Canibais()
    problema.gerar_solucao()
    # Exibe a solução em tela, separando cada passo
    for state in problema.solucao:
        print (state)
        print (34 * '-')

if __name__ == '__main__':
    main()